In [1]:
USER = 'Nunzio'
LOSS = 'mean_euclidean_error'
METRIC = 'MEE'

In [ ]:
#pip install ipykernel==6.26

In [11]:
import pkg_resources

# Ottieni la versione di ipykernel
version = pkg_resources.get_distribution("ipykernel").version

# Stampa la versione
print(f'La versione di ipykernel è: {version}')


La versione di ipykernel è: 6.26.0


In [2]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = '/content/drive/Shareddrives/Project_ML_23/' + USER + '/machine-learning-project'
    sys.path.insert(0,BASE_PATH)
    N_JOBS = -1
    sys.path.insert(0,BASE_PATH + '/src/utils')
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'cup2023','ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'cup2023')
else :
    N_JOBS = -1
    TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [3]:
if (colab := 'google.colab' in sys.modules):
    sys.path.append(BASE_PATH + '/src/utils')
else:
    sys.path.append('../utils')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')
from keras import backend as K


from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2, set_random_state, mean_squared_error
from NN import NeuralNetwork
from joblib import dump

set_random_state(42)

/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') # casting
df_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [5]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target_x,target_y,target_z
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [6]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

# TODO split potenzialmente variabile?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model

In [7]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', NeuralNetwork(loss=LOSS, verbose=0))
])

In [8]:
param_grid = {
    'neuralnetwork__architecture': [
        (128, 64),
        (128, 128),
        (256, 128)
    ],
    'neuralnetwork__activation': ['relu'],
    'neuralnetwork__learning_rate': [1e-3, 1e-2],
    'neuralnetwork__momentum': [0.5,  1.0],
    'neuralnetwork__use_nesterov': [True, False],
    'neuralnetwork__weight_decay': [1e-3, 1e-2],
    'neuralnetwork__input_dimension': [10],
    'neuralnetwork__output_dimension': [3],
    'neuralnetwork__dropout_input_rate': [0.0, 0.1],
    'neuralnetwork__dropout_hidden_rate': [
        (0.1, 0.1),
        (0.2, 0.2)
    ],
    'neuralnetwork__patience': [9],
    'neuralnetwork__epochs': [400],
    'neuralnetwork__batch_size': [16, 32]
}

In [9]:
from sklearn.metrics import make_scorer

mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer, 'MSE': mse_scorer}

In [10]:
from sklearn.model_selection import GridSearchCV, KFold
cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring,
    refit=METRIC,
    n_jobs=N_JOBS,
    verbose=4
)

grid_search.fit(X_train, y_train)

2024-01-20 21:07:26.724934: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-20 21:07:26.726447: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-20 21:07:26.726603: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-20 21:07:26.727497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-20 21:07:26.728968: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-01-20 21:07:27.282986: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-20 21:07:27.283198: I metal_plugin/src/device/metal_device.cc:296] system

KeyboardInterrupt: 

In [ ]:
results = grid_search.cv_results_

results_df = pd.DataFrame(results)
csv_filename_result = f'NN_Cup_Grid_Results_{LOSS}.csv'
csv_filename_loss = f'NN_Cup_Grid_Loss_{LOSS}.csv'
results_df.to_csv(csv_filename_result, index=False)

In [ ]:
best_params = grid_search.best_params_
best_index = grid_search.best_index_

mee = grid_search.best_score_
r2 = grid_search.cv_results_['mean_test_R2'][best_index]
rmse = grid_search.cv_results_['mean_test_RMSE'][best_index]
mse = grid_search.cv_results_['mean_test_MSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))
print('MSE:', abs(mse))

In [ ]:
final_model = grid_search.best_estimator_

# Learning Curve

In [ ]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)
train_scores, test_scores = [], []

best_params = grid_search.best_params_

# Cross-validation with the best parameters
for train_index, test_index in cv.split(X_train):
    X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]

    pipeline_ = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', NeuralNetwork(input_dim=X_train_1.shape[1], verbose=0, **best_params, loss=LOSS)
     )
])
    
    pipeline_.fit(X_train_1, y_train_1, validation_data=(X_test_1, y_test_1),scoring=scoring)

    train_scores.append(pipeline_.history.history)

In [ ]:
train_scores_df = pd.DataFrame(train_scores)
train_scores_df.to_csv(csv_filename_loss, index=False)

train_scores_df['val_loss']

In [ ]:
loss, val_loss, mse, val_mse = [], [], [], []

for i in range(len(train_scores_df)):
    loss.append(train_scores_df.iloc[i]['loss'])
    val_loss.append(train_scores_df.iloc[i]['val_loss'])
    mse.append(train_scores_df.iloc[i]['mean_squared_error'])
    val_mse.append(train_scores_df.iloc[i]['val_mean_squared_erorr'])

df_loss = pd.DataFrame(loss).T
df_val_loss = pd.DataFrame(val_loss).T
df_mse = pd.DataFrame(mse).T
df_val_mse = pd.DataFrame(val_mse).T

mean_loss = df_loss.mean(axis=1)
var_loss = df_loss.var(axis=1)
mean_val_loss = df_val_loss.mean(axis=1)
var_val_loss = df_val_loss.var(axis=1)

mean_mse = df_mse.mean(axis=1)
var_mse = df_mse.var(axis=1)
mean_val_mse = df_val_mse.mean(axis=1)
var_val_mse = df_val_mse.var(axis=1)

plt.figure(figsize=(12, 5))

# loss and val_loss
plt.subplot(1, 2, 1)
plt.plot(mean_loss, label='Loss(MEE)', linestyle='--', grid = True)
plt.fill_between(range(len(mean_loss)), mean_loss-var_loss, mean_loss+var_loss, alpha=0.2)
plt.plot(mean_val_loss, label='Val Loss', linestyle='-#', grid = True)
plt.fill_between(range(len(mean_val_loss)), mean_val_loss-var_val_loss, mean_val_loss+var_val_loss, alpha=0.2)
plt.title('Learning Curve for Loss (MEE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# mse and val mse
plt.subplot(1, 2, 2)
plt.plot(mean_mse, label='MSE', linestyle='-', grid = True)
plt.fill_between(range(len(mean_mse)), mean_mse-var_mse, mean_mse+var_mse, alpha=0.2)
plt.plot(mean_val_mse, label='Val MSE', linestyle='-*')
plt.fill_between(range(len(mean_val_mse)), mean_val_mse-var_val_mse, mean_val_mse+var_val_mse, alpha=0.2)
plt.title('Learning Curve for MSE')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()

plt.tight_layout()
save_plot(plt, IMAGES_FOLDER, f'learning_curve_nn_cup_grid_{METRIC}', extension='png')
plt.show()

# Test of the model

In [ ]:
y_pred = final_model.predict(X_test)

mee = mean_euclidean_error(y_test, y_pred)
r2 = multidim_r2(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('MEE:', mee)
print('R2:', r2)
print('RMSE:', rmse)
print('MSE:', mse)

# Save model

In [ ]:
# qui fare refit finale sul modello compreso il test set

X = np.concatenate((X_train, X_test))
Y = np.concatenate((y_train, y_test))

pipeline_test = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', NeuralNetwork(input_dim=X, verbose=0, **best_params, loss='mean_euclidean_error'))
])
pipeline_test.fit(X, Y, scoring=scoring)




In [ ]:
# Save the model best parameters
best_params = grid_search.best_params_
best_params_df = pd.DataFrame([best_params])
csv_filename = f'best_params_Grid_NN_{METRIC}.csv'
best_params_df.to_csv(csv_filename, index=False)


In [ ]:
# Save the model
model_path = os.path.join(MODEL_FOLDER, f'NN_model_grid_NN_{METRIC}.joblib')
model_path_pkl = os.path.join(MODEL_FOLDER, f'NN_model_grid_NN_pkl_{METRIC}.pkl')
dump(pipeline_test, model_path, compress=3)
dump(pipeline_test, model_path_pkl, compress=3)
